# Generator notebook

This notebook is purely to illustrate how the data in this directory is generated.

In [3]:
import numpy as np

## Definition of potential and simulation functions

In [4]:
# assymetric double well potential 
def potential(x):
    return x - 2.0 - 3.0 * (x - 2.0)**2 + 0.5 * (x - 2.0)**4

# gradient of the potential
def gradient(x):
    return 2.0 * x**3 - 12.0 * x**2 + 18.0 * x - 3.0

# harmonic bias potential
def bias_potential(x, k, x0):
    return 0.5 * k * (x - x0)**2

# gradient of the harmonic bias potential
def bias_gradient(x, k, x0):
    return k * (x - x0)

In [5]:
def new_temperature_idx(old_idx, epot, kTs, f):
    new_idx = np.random.randint(kTs.shape[0])
    delta = (1.0 / kTs[new_idx] - 1.0 / kTs[old_idx]) * epot - f[new_idx] + f[old_idx]
    if delta <= 0.0:
        return new_idx
    elif np.random.rand() < np.exp(-delta):
        return new_idx
    else:
        return old_idx
    
#Brownian Dynamics simulated tempering
def bd_st(
    x0, length_between_changes, number_of_changes, kTs, f,
    temperature_idx=0, mass=1.0, damping=1.0, dt=0.005):
    coeff_A = dt / (mass * damping)
    coeff_B = np.sqrt(2.0 * dt * kTs[temperature_idx] / (mass * damping))
    x = [x0]
    kT = [kTs[temperature_idx]]
    u = [potential(x0)]
    for _i in range(number_of_changes):
        for _j in range(length_between_changes):
            x.append(x[-1] - coeff_A * gradient(x[-1]) + coeff_B * np.random.normal())
            kT.append(kTs[temperature_idx])
            u.append(potential(x[-1]))
        temperature_idx = new_temperature_idx(temperature_idx, u[-1], kTs, f)
        coeff_B = np.sqrt(2.0 * dt * kTs[temperature_idx] / (mass * damping))
    conf_traj = np.array(x, dtype=np.float64).reshape((-1, 1))
    temp_traj = np.array(kT, dtype=np.float64)
    energy_traj = np.array(u, dtype=np.float64) / temp_traj
    return conf_traj, temp_traj, energy_traj

#Brownian Dynamics umbrella sampling
def bd_us(
    x0, length, k_bias, x0_bias,
    mass=1.0, damping=1.0, dt=0.005, kT=1.0):
    coeff_A = dt / (mass * damping)
    coeff_B = np.sqrt(2.0 * dt * kT / (mass * damping))
    x = [x0]
    for _i in range(length):
        x.append(
            x[-1] - coeff_A * (gradient(x[-1]) + bias_gradient(x[-1], k_bias, x0_bias)) + coeff_B * np.random.normal())
    xtraj = np.array(x, dtype=np.float64).reshape((-1, 1))
    return xtraj

## Generate simulated tempering data

In [6]:
# kT values to use
kTs = np.array([1.0, 2.0, 3.5, 7.5], dtype=np.float64)

# precalculated free energies of the thermodynamic states in kT
f_therm = np.array([0.0, 2.58394215, 3.45680378, 3.91820844], dtype=np.float64)

# generate and store trajectories
for i in range(20):
    conf_traj, temp_traj, energy_traj = bd_st(np.random.rand() * 5.0 - 0.5, 100, 99, kTs, f_therm)
    np.save("multi_temperature/conf_traj_%03d.npy" % i, conf_traj)
    np.save("multi_temperature/temp_traj_%03d.npy" % i, temp_traj)
    np.save("multi_temperature/energy_traj_%03d.npy" % i, energy_traj)

# Generate umbrella sampling data

In [ ]:
# generate umbrella parameters
n_umbrellas = 20
independent_simulations_per_state = 5
k_bias = np.ones(shape=(n_umbrellas,), dtype=np.float64) * 30.0
x0_bias = np.zeros(shape=(n_umbrellas,), dtype=np.float64)
x0_bias = np.linspace(-0.5, 4.5, n_umbrellas)
length = 1000

# generate and store umbrella sampling trajectories
j = 0
fh = open("umbrella_sampling/meta.dat", 'w')
fh.write("# index umbrella_center force_constant\n")
for i in range(independent_simulations_per_state):
    for K in range(n_umbrellas):
        np.save("umbrella_sampling/us_traj_%03d.npy" % j, bd_us(x0_bias[K], 1000, k_bias[K], x0_bias[K]))
        fh.write("%7d %15.8f %14.8f\n" % (j, x0_bias[K], k_bias[K]))
        j += 1
fh.close()

# generate and store unbiased trajectories
for i in range(5):
    np.save("umbrella_sampling/md_traj_%03d.npy" % i, bd_us(np.random.rand() * 5.0 - 0.5, 20000, 0.0, 0.0))